# Importing pyspark and the json files 

In [1]:
# import findspark
#import findspark

# initialize findspark with spark directory

#ALWAYS HAVE TO BE CHANGED 
#path = "/Users/konstantinlazarov/Desktop/Big_Data/PySpark/Week_5/spark"
#findspark.init(path) 

# import pyspark
import pyspark
# create spark context
sc = pyspark.SparkContext()
# create spark session 
spark = pyspark.sql.SparkSession(sc)

22/11/21 13:17:53 WARN Utils: Your hostname, Konstantins-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.45 instead (on interface en0)
22/11/21 13:17:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/21 13:17:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark

In [3]:
#set this path to your path, for some reason I have an error 
#reading in all the files
import os 
path_json = ".././../data/Topic_vegan/*.json"

In [4]:
df_json = spark.read.option("multiline","true").json(path_json)

22/11/21 13:18:39 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [5]:
df_json.count()

3428559

# Question 1 The number of followers of the brand 

Brand defined as vegan? 

In [6]:
# select interesting features
import pyspark.sql.functions as F

df_json_sub = df_json.select(F.col("user.name"),
                                F.col("user.screen_name"),
                                F.col("created_at"), 
                                F.col("full_text"),
                                F.col("entities.hashtags"),
                                F.col("lang"),
                                F.col("favorite_count"),
                                F.col("retweet_count"),
                                F.col("user.followers_count"),
                                F.col("user.friends_count"),
                                F.col("user.favourites_count"),
                                F.col("entities.urls"),
                                F.col("entities.symbols"))

df_json_sub.limit(5).toPandas()

,name,screen_name,created_at,full_text,hashtags,lang,favorite_count,retweet_count,followers_count,friends_count,favourites_count,urls,symbols
0,のり/Nori,nori_k_629,Mon Apr 04 10:09:55 +0000 2022,RT @ohmpawatt: เพื่อนๆคิดถึงผมมั้ยยย ถ้าคิดถึง...,[],th,0,3582,139,346,63382,[],[]
1,alice,myn4meizalize,Mon Apr 04 10:09:54 +0000 2022,RT @mynameisnanon: คิดถึงกันป่าว ถ้าคิดถึงต้อง...,[],th,0,3837,655,194,8726,[],[]
2,Karen Reed 🌸,kandk670,Mon Apr 04 10:09:54 +0000 2022,@trudiebakescake Organic coconut oil in a jar ...,[],en,0,0,711,429,3420,[],[]
3,ハル):),patlnwza55,Mon Apr 04 10:09:52 +0000 2022,RT @ohmpawatt: เพื่อนๆคิดถึงผมมั้ยยย ถ้าคิดถึง...,[],th,0,3582,236,187,15655,[],[]
4,alice,myn4meizalize,Mon Apr 04 10:09:52 +0000 2022,RT @ohmpawatt: เพื่อนๆคิดถึงผมมั้ยยย ถ้าคิดถึง...,[],th,0,3582,655,194,8726,[],[]


In [7]:
#doing basic pre-processing 

import pandas as pd 
#drop duplicates  
df_json_sub = df_json_sub.filter(~F.col("full_text").startswith("RT"))\
                        .drop_duplicates().cache()

In [8]:
pd.set_option('display.max_colwidth', None)
df_json_sub.select("created_at", "full_text").limit(5).toPandas()

,created_at,full_text
0,Mon Oct 18 09:24:09 +0000 2021,"Sponsor #1: https://t.co/ixCQVKJZG5 After the campaign, their organic traffic rose from 29.5k to 46k per month. 35.9% https://t.co/3G4iPPmO7Z"
1,Mon Apr 04 10:47:40 +0000 2022,@tewfootprints #GoGreen #Recycle #Organic
2,Mon Apr 04 10:47:03 +0000 2022,@GillwallyO @irish_organic @peatyGHG This + if you can get some hay locally that is full of natural seeds.\n\nWe have done this on a roughly 50 × 50m plot with 20 layers &amp; a bunch of broilers. Cracking resukts
3,Mon Apr 04 10:43:32 +0000 2022,Great River Organic Milling Great River Organic Whole Corn 25 Pounds Pack of https://t.co/aahZzoPJ6C eBay https://t.co/az0HFosgSi
4,Tue Sep 13 19:13:49 +0000 2022,"🥕NEW Vegin' Out September 19th Vegan Delivery Menu!\n\n🥑#Vegan Meal plan arrives with 3 entrees, 4 sides, soup &amp; dessert - about 5 meals for two! \n\n😍Take a break from cooking!\n\nOrder Today!\n👉https://t.co/4LkIUjjA3F\n\n#govegan #fitness #goals #ビーガン #비건 #organic #weightloss https://t.co/mJvDVOKfZu"


In [9]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [10]:

df_users1 = df_json_sub.groupBy('name')\
                .agg(countDistinct('created_at'))\
                .withColumnRenamed('count(created_at)', 'count_tweets')\
                .sort('count_tweets', descending = False)\
                .cache()

In [11]:
df_users1.printSchema()

root
 |-- name: string (nullable = true)
 |-- count_tweets: long (nullable = false)



In [12]:
df_users1.show(5)

22/11/21 13:19:08 WARN MemoryStore: Not enough space to cache rdd_24_99 in memory! (computed 3.9 MiB so far)
22/11/21 13:19:08 WARN BlockManager: Persisting block rdd_24_99 to disk instead.
22/11/21 13:19:08 WARN MemoryStore: Not enough space to cache rdd_24_100 in memory! (computed 3.9 MiB so far)
22/11/21 13:19:08 WARN MemoryStore: Failed to reserve initial memory threshold of 1024.0 KiB for computing block rdd_24_105 in memory.
22/11/21 13:19:08 WARN BlockManager: Persisting block rdd_24_100 to disk instead.
22/11/21 13:19:08 WARN MemoryStore: Not enough space to cache rdd_24_105 in memory! (computed 384.0 B so far)
22/11/21 13:19:08 WARN BlockManager: Persisting block rdd_24_105 to disk instead.
22/11/21 13:19:08 WARN MemoryStore: Not enough space to cache rdd_24_99 in memory! (computed 3.9 MiB so far)


22/11/21 13:19:08 WARN MemoryStore: Not enough space to cache rdd_24_107 in memory! (computed 3.9 MiB so far)
22/11/21 13:19:08 WARN BlockManager: Persisting block rdd_24_107 to disk instead.
22/11/21 13:19:08 WARN MemoryStore: Not enough space to cache rdd_24_109 in memory! (computed 3.9 MiB so far)
22/11/21 13:19:08 WARN BlockManager: Persisting block rdd_24_109 to disk instead.


22/11/21 13:19:09 WARN MemoryStore: Not enough space to cache rdd_24_117 in memory! (computed 3.9 MiB so far)
22/11/21 13:19:09 WARN BlockManager: Persisting block rdd_24_117 to disk instead.


22/11/21 13:19:09 WARN MemoryStore: Not enough space to cache rdd_24_125 in memory! (computed 3.9 MiB so far)
22/11/21 13:19:09 WARN BlockManager: Persisting block rdd_24_125 to disk instead.
22/11/21 13:19:09 WARN MemoryStore: Not enough space to cache rdd_24_125 in memory! (computed 3.9 MiB so far)


22/11/21 13:19:09 WARN MemoryStore: Not enough space to cache rdd_24_130 in memory! (computed 3.9 MiB so far)
22/11/21 13:19:09 WARN BlockManager: Persisting block rdd_24_130 to disk instead.
22/11/21 13:19:09 WARN MemoryStore: Not enough space to cache rdd_24_131 in memory! (computed 3.9 MiB so far)
22/11/21 13:19:09 WARN BlockManager: Persisting block rdd_24_131 to disk instead.
22/11/21 13:19:10 WARN MemoryStore: Not enough space to cache rdd_24_130 in memory! (computed 3.9 MiB so far)


...

22/11/21 13:19:11 WARN MemoryStore: Not enough space to cache rdd_24_159 in memory! (computed 3.9 MiB so far)
[78.796s][warning][gc,alloc] Executor task launch worker for task 160.0 in stage 9.0 (TID 4510): Retried waiting for GCLocker too often allocating 30642 words
[78.798s][warning][gc,alloc] Executor task launch worker for task 158.0 in stage 9.0 (TID 4508): Retried waiting for GCLocker too often allocating 256 words
[78.798s][warning][gc,alloc] Executor task launch worker for task 162.0 in stage 9.0 (TID 4512): Retried waiting for GCLocker too often allocating 16952 words
[78.798s][warning][gc,alloc] Executor task launch worker for task 161.0 in stage 9.0 (TID 4511): Retried waiting for GCLocker too often allocating 256 words
[78.812s][warning][gc,alloc] Executor task launch worker for task 162.0 in stage 9.0 (TID 4512): Retried waiting for GCLocker too often allocating 140002 words
22/11/21 13:19:11 WARN BlockManager: Block rdd_24_162 could not be removed as it was not found on 

Exception in thread "Executor task launch worker for task 164.0 in stage 9.0 (TID 4514)" java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.memory.TaskMemoryManager.<init>(TaskMemoryManager.java:93)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:486)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1136)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:635)
	at java.base/java.lang.Thread.run(Thread.java:833)


RDD.scala:376)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:327)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:365)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:329)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:365)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:329)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:365)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:329)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:365)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:329)
22/11/21 13:19:11 WARN TaskSetManager: Lost task 162.0 in stage 9.0 (TID 4512) (192.168.1.45 executor driver): java.lang.OutOfMemoryError: J

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.10/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/homebrew/lib/python3.10/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/homebrew/opt/ipython/libexec/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, se

ConnectionRefusedError: [Errno 61] Connection refused

In [ ]:
df_users = df_users1.toPandas()
df_users


In [ ]:
df_screen_name = df_json_sub.groupBy('screen_name')\
                .agg(countDistinct('created_at'))\
                .withColumnRenamed('count(created_at)', 'count_tweets')\
                .sort('count_tweets', descending = False)\
                .cache()

In [ ]:
df_screen_name.show(5)

# Question 2: The number of likes for a brand post 

# Question 3: The number and impact of influencers for each brand 